In [4]:
# Run from Model Classes
import importlib
from src import elo_models as EloModels
importlib.reload(EloModels)

# Game Inputs
elo_h_base = 1502  # Home Team
elo_a_base = 1471  # Away Team
result_h = 34  # Home Team Points
result_a = 33  # Away Team Points
print('Model Inputs:')
print(f'Home Team Pregame ELO = {elo_h_base}')
print(f'Home Team Final Score = {34}')
print(f'Away Team Pregrame ELO = {elo_a_base}')
print(f'Away Team Final Score = {result_a}')
print()


# Set Up 538 ELO Model and get vegas line
print('--- 538 Model ---')
elo_538 = EloModels.FiveThirtyEightElo(
    home_field_add=55,
    home_field_dist_add=4,
    rest_add=25,
    playoff_mult=1.2,
    k=20
)

# 538 ELO Adjustments
elo_h_adj = elo_538.elo_adjustment(
    elo_base=elo_h_base,
    is_home_team=True,
    travel_dist=0,
    coming_off_bye=False,
    is_playoffs=False
)
elo_a_adj = elo_538.elo_adjustment(
    elo_base=elo_a_base,
    is_home_team=False,
    travel_dist=0,
    coming_off_bye=False,
    is_playoffs=False
)

# Get Probability of home win, Vegas Line
home_w_prob, vegas_line_538 = elo_538.get_vegas_line(elo_h_adj, elo_a_adj)
print(f"538's Model Vegas Line = {vegas_line_538}")

# Update ELO Ratings for both teams for the 538 Model
elo_h_post_game, elo_a_post_game = elo_538.get_post_game_elos(
    elo_h_pregame=elo_h_base,
    elo_a_pregame=elo_a_base,
    h_score=result_h,
    a_score=result_a,
    h_win_prob=home_w_prob
)
print(f'Home Team ELO changed from {elo_h_base} to {elo_h_post_game}')
print(f'Away Team ELO changed from {elo_a_base} to {elo_a_post_game}')
print()

# Set Up Johns Model and get vegas line
elo_john = EloModels.JohnElo(
    coeff=0.075,
    exp_ratio=(699/999),
    home_field_add=1.5
)
vegas_line_john = elo_john.get_vegas_line(elo_h_base, elo_a_base)
print("--- John's Model ---")
print(f"John's Model Vegas Line = {vegas_line_john}")

# Update ELO Ratings for both teams for Johns Model
# Model's Updating Properties

margin_pct = .02
h_a_pct = .95

# Coeff A
if result_h > result_a:
    coeff_a = margin_pct*h_a_pct*elo_a_base
else:
    coeff_a = -margin_pct*h_a_pct*elo_h_base

# Coeff B
if result_h > result_a:
    if elo_h_base > elo_a_base:
        coeff_b = 1-(elo_h_base-elo_a_base)/elo_h_base
    elif elo_a_base > elo_h_base:
        coeff_b = 1+(elo_a_base-elo_h_base)/elo_a_base
    else:
        coeff_b = 1
else:
    if elo_h_base > elo_a_base:
        coeff_b = 1+(elo_h_base-elo_a_base)/elo_h_base
    elif elo_a_base > elo_h_base:
        coeff_b = 1-(elo_a_base-elo_h_base)/elo_a_base
    else:
        coeff_b = 1

# Run Calculation
home_team_elo_change = coeff_a*coeff_b**0.25
elo_h_post_game = elo_h_base + home_team_elo_change
elo_a_post_game = elo_a_base - home_team_elo_change
print(f'Home Team ELO changed from {elo_h_base} to {elo_h_post_game}')
print(f'Home Team ELO changed from {elo_a_base} to {elo_a_post_game}')

Model Inputs:
Home Team Pregame ELO = 1502
Home Team Final Score = 34
Away Team Pregrame ELO = 1471
Away Team Final Score = 33

--- 538 Model ---
538's Model Vegas Line = -3.44
Home Team ELO changed from 1502 to 1507.176993626179
Away Team ELO changed from 1471 to 1465.823006373821
John's Model Vegas Line = -2.329023025091661


27.803659336825614

In [ ]:
### Evaluate Johns Model Performance for 2021


